# MAA307 Homework Assignment 2

#### Lucien Walewski and Aurele Bohbot

In [1]:
# Imports

import numpy as np
import matplotlib.pyplot as plt

## Exercise 6.11 A

We place ourselves in the framework of Exercise 6.8. We consider the problem
$$
\text{minimize}_{x\in\mathbb{R}^n} \frac{1}{2}\langle Ax,x\rangle - \langle b,x\rangle \quad\text{subject to} \quad Cx=d
$$
where $A\in\mathcal{M}_n(\mathbb{R})$ is symmetric positive definite, $b\in\mathbb{R}^n$, $C\in\mathcal{M}_{m,n}(\mathbb{R})$ is surjective, and $d\in\mathbb{R}^m$. We begin by considering the associated Lagrangian/dual problem given by:
$$
L(x, \lambda) = \frac{1}{2}\langle Ax,x\rangle - \langle b,x\rangle + \langle \lambda,Cx\rangle - \langle \lambda,d\rangle
$$
for $(x,\lambda)\in X\times\mathbb{R}^m$. 

Then, Uzawa's algorithm consists of the following:
\begin{align*}
    x^k\in\argmin_{x\in X} L(x, \lambda^k) \\
    \lambda^{k+1}=\lambda^k+\tau (Cx^k-d)
\end{align*}
We can explicitely compute the update step for $x_k$. We have
\begin{align*}
    x^k\in\argmin_{x\in X} L(x, \lambda^k)&\iff \nabla_x L(x^k, \lambda^k)=0 \\
    &\iff 2Ax^k-b+C^t\lambda^k =0 \\
    &\iff x^k=A^{-1}(b-C^t\lambda^k)
\end{align*}

We now implement the Uzawa algorithm for this problem.

In [26]:
def Uzawa(A, b, C, d, stepsize, tol):
    # Initialization
    lambda_ = np.zeros(d.shape)
    invA = np.linalg.inv(A)
    x = invA @ (b - C @ lambda_)
    lambda_ = lambda_ + stepsize * (C @ x -d)
    while True:
        # Update
        x_new = invA @ (b - C @ lambda_)
        lambda_ = lambda_ + stepsize * (C @ x_new - d)
        # Stopping criterion
        if np.linalg.norm(x_new - x) < tol:
            return x_new, lambda_
        x = x_new

In [27]:
# Example values

A = np.array([[2, 1], [1, 1]]) # Symmetric positive definite
C = np.array([[1,2], [3,1]]) # Surjective
d = np.array([3,1])
b = np.array([1,1])
τ = 0.1
tol = 1e-6

x, lambda_ = Uzawa(A, b, C, d, τ, tol)